In [57]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython
%load_ext autotime

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
import pandas as pd
import numpy as np
import scipy as sp
from GELMMnet import GELMMnet, kinship
print(nb_dir)

from sklearn.metrics import mean_squared_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet
time: 13.8 ms


In [18]:
%%R -o out,x,y,n,p
library(gelnet)
library(selectiveInference)
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
print(beta)
y = x%*%beta + sigma*rnorm(n)
y=scale(y,TRUE,FALSE)

out = gelnet(x, y, 1.0, 1.0, nFeats = NULL, a = rep(1, n), d = rep(1, p),
      P = diag(p), m = rep(0, p), max.iter = 100, eps = 1e-05,
      w.init = rep(0, p), b.init = NULL, fix.bias = FALSE, silent = FALSE,
      balanced = FALSE, nonneg = FALSE)
out

 [1] 3 2 0 0 0 0 0 0 0 0
Training a linear regression model
Running linear regression optimization with L1 = 1.000000, L2 = 1.000000
f = 5.095377 after iteration 3
$w
 [1] 0.8211052 0.3338258 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.0000000 0.0000000 0.0000000

$b
[1] 0



time: 13.4 ms


In [59]:
from GELMMnet import kinship, GELMMnet, normalized_laplacian, transform_kernel_to_distance


from sklearn.preprocessing import scale
n = 50
p = 10
sigma = 1
beta = np.zeros(shape=p)
beta[0] = 3
beta[1] = 2
P=np.identity(int(p))
l1=1.0;l2=1.0



time: 11.7 ms


In [61]:
from GELMMnet import kinship, GELMMnet, normalized_laplacian, transform_kernel_to_distance


#x = np.random.rand(n,p)
#x=scale(x)
#y = np.dot(x,beta) + sigma*np.random.normal(size=n)
#y=scale(y, with_std=False)
print (np.mean(y))
print(np.mean(x, axis=0))
g = GELMMnet(y, x, np.identity(int(n)))
print(dir(g))
g.fit(P,lmm=False,l1=l1,l2=l2)
g.predict(x)
#g.post_selection_analysis(lmm=False, alpha=0.05, compute_intervals=True)
    

    #x = np.random.rand(n,p)
    #x=scale(x)
    #y = np.dot(x,beta) + sigma*np.random.normal(size=n)
#GELMMnet(y, x, np.identity(int(n))).train(P,lmm=False,l1=l1,l2=l2)
#print("Old train")
#%timeit old_train()
#
#print("\n reger train")
#%timeit train_regerg()


0.0549614146158
[ -8.67361738e-18  -1.77635684e-17  -2.22044605e-17  -3.35842465e-17
  -3.60822483e-18  -1.11022302e-17   2.77555756e-17  -1.77635684e-17
  -1.88737914e-17   1.60982339e-17]
['Xtilde', '_GELMMnet__K', '_GELMMnet__P', '_GELMMnet__SUX', '_GELMMnet__SUy', '_GELMMnet__X', '_GELMMnet__b', '_GELMMnet__isIntercept', '_GELMMnet__isStandardized', '_GELMMnet__islmm', '_GELMMnet__keep', '_GELMMnet__l1', '_GELMMnet__l2', '_GELMMnet__ldelta', '_GELMMnet__m', '_GELMMnet__n', '_GELMMnet__sigma', '_GELMMnet__summary', '_GELMMnet__w', '_GELMMnet__y', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'b', 'ldelta', 'post_selection_analysis', 'predict', 'set_sigma', 'sigma', 'summary', 'w', 'ytilde']


AttributeError: 'GELMMnet' object has no attribute 'fit'

time: 28.9 ms


In [19]:
from GELMMnet import kinship, GELMMnet, normalized_laplacian, transform_kernel_to_distance
import scipy as SP

geno_filename = os.path.join(nb_dir,'./test/genotypes.csv')
X = SP.genfromtxt(geno_filename)
y = SP.genfromtxt(os.path.join(nb_dir,"test/phenotypes.csv"))
K = kinship(X)
P = np.identity(X.shape[1])
n_s,n_v = X.shape
n_train = 150
n_test = n_s - n_train


# split into training and testing
train_idx = SP.random.permutation(SP.arange(n_s))
test_idx = train_idx[n_train:]
train_idx = train_idx[:n_train]

# X[train_idx],K[train_idx][:,train_idx],y[train_idx]

print("P Dim:",P.shape)
print("X Dim", X.shape)
glm = GELMMnet(y[train_idx], X[train_idx], K[train_idx][:,train_idx])
glm.train(P, l1=1, l2=0.5, max_iter=1000, eps=1.0e-5)

y_pred = glm.predict(X[test_idx])
msr_regreg = mean_squared_error(y[test_idx],y_pred)
w_regreg = glm.w
print("W",w_regreg.sum())
print("MSR:",msr_regreg)

P Dim: (1000, 1000)
X Dim (176, 1000)
W 0.349033827998
MSR: 0.686802035852


In [32]:
%%R -o x,y,out
library(glmnet)
library(selectiveInference)
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
y = x%*%beta + sigma*rnorm(n)
# first run glmnet
#lambdahat = cv.glmnet(x,y,standardize=FALSE)
#print(lambdahat)
#sigmahat = estimateSigma(x,y, standardize=FALSE, intercept=F)$sigmahat

gfit = glmnet(x,y,standardize=FALSE)
# extract coef for a given lambda; note the 1/n factor!
# (and we don't save the intercept term) fixedLassoInf 7 
lambda = .8
args = coef(gfit, s=lambda/n, exact=TRUE)
print(args)
# compute fixed lambda p-values and selection intervals
out = fixedLassoInf(x,y,args[-1],lambda,sigma=sigma)
out

11 x 1 sparse Matrix of class "dgCMatrix"
                      1
(Intercept)  0.05496141
V1           2.90117931
V2           2.09140871
V3           0.10536154
V4           0.07723765
V5          -0.17150755
V6          -0.13135784
V7           0.13166391
V8           0.02265503
V9          -0.09731043
V10          .         

Call:
fixedLassoInf(x = x, y = y, beta = args[-1], lambda = lambda, 
    sigma = sigma)

Standard deviation of noise (specified or estimated) sigma = 1.000

Testing results at lambda = 0.800, with alpha = 0.100

 Var   Coef Z-score P-value LowConfPt UpConfPt LowTailArea UpTailArea
   1  2.912  18.897   0.000     2.655    3.196       0.048      0.049
   2  2.123  13.803   0.000     1.856    2.377       0.049      0.049
   3  0.141   0.817   0.495    -0.728    0.404       0.049      0.049
   4  0.109   0.699   0.578    -0.854    0.337       0.050      0.049
   5 -0.206  -1.240   0.257    -0.486    0.310       0.048      0.049
   6 -0.145  -0.968   0.358    -0.395

time: 51.1 ms


In [52]:
from GELMMnet import GELMMnet
l1=.8/float(n)
P = np.identity(x.shape[1])
g = GELMMnet(y, x)
print(g.train(P,l1=l1,l2=0))

g.post_selection_analysis(alpha=0.1,compute_intervals=True)

(-9.325873406851315e-17, array([ 2.90089583,  2.09186778,  0.10637913,  0.07805434, -0.17215038,
       -0.13101207,  0.13188417,  0.02279718, -0.09728396,  0.        ]))
Active [0 1 2 3 4 5 6 7 8]


/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:305: RuntimeWarning: invalid value encountered in double_scalars
  def _truncatednorm_surv(z, mean, a, b, sd):
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:313: RuntimeWarning: overflow encountered in exp
  if mean == -np.inf:
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:315: RuntimeWarning: overflow encountered in exp
  


,pval,coef0,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
0.0,0.0,2.911576,2.900896,1.063790e+18,2.736984e-16,inf,0.0,0.0,2.736984e-18
1.0,0.0,2.122611,2.091868,7.770508e+17,2.731624e-16,inf,0.0,0.0,2.731624e-18
2.0,0.0,0.141078,0.106379,4.596589e+16,3.069196e-16,inf,0.0,0.0,3.069196e-18
3.0,0.0,0.109437,0.078054,3.936711e+16,2.779917e-16,inf,0.0,0.0,2.779917e-18
4.0,0.0,-0.205650,-0.172150,-6.981093e+16,-inf,-2.945816e-16,0.0,0.0,2.945816e-18
5.0,0.0,-0.144647,-0.131012,-5.448261e+16,-inf,-2.654913e-16,0.0,0.0,2.654913e-18
6.0,0.0,0.141072,0.131884,5.116789e+16,2.757051e-16,inf,0.0,0.0,2.757051e-18
7.0,0.0,0.045156,0.022797,1.711204e+16,2.638865e-16,inf,0.0,0.0,2.638865e-18
8.0,0.0,-0.112833,-0.097284,-4.066726e+16,-inf,-2.774537e-16,0.0,0.0,2.774537e-18


time: 2.59 s


In [6]:
%%R
library(glmnet)
library(selectiveInference)

# Returns Prob(Z>z | Z in [a,b]), where mean can be a vector

tnorm.surv <- function(z, mean, sd, a, b, bits=NULL) {
  z = max(min(z,b),a)
  
  # Check silly boundary cases
  p = numeric(length(mean))
  p[mean==-Inf] = 0
  p[mean==Inf] = 1
  
  # Try the multi precision floating point calculation first
  o = is.finite(mean)
  mm = mean[o]
  pp = mpfr.tnorm.surv(z,mm,sd,a,b,bits)
  # If there are any NAs, then settle for an approximation
  oo = is.na(pp)
  if (any(oo)) pp[oo] = bryc.tnorm.surv(z,mm[oo],sd,a,b)
    
  p[o] = pp
  #print(paste("z",z,"mean",mean,"sd",sd,"a",a,"b",b,"pval",p))
  return(p)
}

# Returns Prob(Z>z | Z in [a,b]), where mean cane be a vector, using
# multi precision floating point calculations thanks to the Rmpfr package

mpfr.tnorm.surv <- function(z, mean=0, sd=1, a, b, bits=NULL) {
  # If bits is not NULL, then we are supposed to be using Rmpf
  # (note that this was fail if Rmpfr is not installed; but
  # by the time this function is being executed, this should
  # have been properly checked at a higher level; and if Rmpfr
  # is not installed, bits would have been previously set to NULL)
  #if (!is.null(bits)) {
  #  z = Rmpfr::mpfr((z-mean)/sd, precBits=bits)
  #  a = Rmpfr::mpfr((a-mean)/sd, precBits=bits)
  #  b = Rmpfr::mpfr((b-mean)/sd, precBits=bits)
  #  return(as.numeric((Rmpfr::pnorm(b)-Rmpfr::pnorm(z))/
  #                    (Rmpfr::pnorm(b)-Rmpfr::pnorm(a))))
  #}
  
  # Else, just use standard floating point calculations
  z = (z-mean)/sd
  a = (a-mean)/sd
  b = (b-mean)/sd
  return((pnorm(b)-pnorm(z))/(pnorm(b)-pnorm(a)))
}

# Returns Prob(Z>z | Z in [a,b]), where mean can be a vector, based on
# A UNIFORM APPROXIMATION TO THE RIGHT NORMAL TAIL INTEGRAL, W Bryc
# Applied Mathematics and Computation
# Volume 127, Issues 23, 15 April 2002, Pages 365--374
# https://math.uc.edu/~brycw/preprint/z-tail/z-tail.pdf

bryc.tnorm.surv <- function(z, mean=0, sd=1, a, b) {
  z = (z-mean)/sd
  a = (a-mean)/sd
  b = (b-mean)/sd
  n = length(mean)

  term1 = exp(z*z)
  o = a > -Inf
  term1[o] = ff(a[o])*exp(-(a[o]^2-z[o]^2)/2)
  term2 = rep(0,n)
  oo = b < Inf
  term2[oo] = ff(b[oo])*exp(-(b[oo]^2-z[oo]^2)/2)
  p = (ff(z)-term2)/(term1-term2)

  # Sometimes the approximation can give wacky p-values,
  # outside of [0,1] ..
  #p[p<0 | p>1] = NA
  old = p
  p = pmin(1,pmax(0,p))

  return(p)
}

ff <- function(z) {
  return((z^2+5.575192695*z+12.7743632)/
         (z^3*sqrt(2*pi)+14.38718147*z*z+31.53531977*z+2*12.77436324))
}

# Return Prob(Z>z | Z in [a,b]), where mean can be a vector, based on
# Riemann approximation tricks, by Max G'Sell

gsell.tnorm.surv <- function(z, mean=0, sd=1, a, b) {
  return(max.approx.frac(a/sd,b/sd,z/sd,mean/sd))
}

    
pinv <- function(A, tol=.Machine$double.eps) {
  e = eigen(A)
  v = Re(e$vec)
  d = Re(e$val)
  d[d > tol] = 1/d[d > tol]
  d[d < tol] = 0
  if (length(d)==1) return(v*d*v)
  else return(v %*% diag(d) %*% t(v))
}

      
poly.pval <- function(y, G, u, v, sigma, bits=NULL) {
  z = sum(v*y)
  vv = sum(v^2)
  sd = sigma*sqrt(vv)
  
  rho = G %*% v / vv
 
  vec = (u - G %*% y + rho*z) / rho
  
  vlo = suppressWarnings(max(vec[rho>0]))
  vup = suppressWarnings(min(vec[rho<0]))

  pv = tnorm.surv(z,0,sd,vlo,vup,bits)
  print(pv)
  return(list(pv=pv,vlo=vlo,vup=vup))
}
      
      
poly.int <- function(y, G, u, v, sigma, alpha, gridrange=c(-100,100),
                     gridpts=100, griddepth=2, flip=FALSE, bits=NULL) {
  
  z = sum(v*y)
  vv = sum(v^2)
  sd = sigma*sqrt(vv)
  
  rho = G %*% v / vv
  vec = (u - G %*% y + rho*z) / rho
  vlo = suppressWarnings(max(vec[rho>0]))
  vup = suppressWarnings(min(vec[rho<0]))
  
  xg = seq(gridrange[1]*sd,gridrange[2]*sd,length=gridpts) 
  
  fun = function(x) { tnorm.surv(z,x,sd,vlo,vup,bits) }

  int = grid.search(xg,fun,alpha/2,1-alpha/2,gridpts,griddepth)
 
  tailarea = c(fun(int[1]),1-fun(int[2]))

  if (flip) {
    int = -int[2:1]
    tailarea = tailarea[2:1]
  }
  
  return(list(int=int,tailarea=tailarea))
}

##############################

# Assuming that grid is in sorted order from smallest to largest,
# and vals are monotonically increasing function values over the
# grid, returns the grid end points such that the corresponding
# vals are approximately equal to {val1, val2}

grid.search <- function(grid, fun, val1, val2, gridpts=100, griddepth=2) {
  n = length(grid)
  vals = fun(grid)
  
  print(vals)
    
  ii = which(vals >= val1)
  jj = which(vals <= val2)
  if (length(ii)==0) return(c(grid[n],Inf))   # All vals < val1
  if (length(jj)==0) return(c(-Inf,grid[1]))  # All vals > val2
  # RJT: the above logic is correct ... but for simplicity, instead,
  # we could just return c(-Inf,Inf) 
 
  i1 = min(ii); i2 = max(jj)
  print(paste("i2 ", i2, val2))
  if (i1==1) lo = -Inf
  else lo = grid.bsearch(grid[i1-1],grid[i1],fun,val1,gridpts,
         griddepth-1,below=TRUE)
  if (i2==n) hi = Inf
  else hi = grid.bsearch(grid[i2],grid[i2+1],fun,val2,gridpts,
         griddepth-1,below=FALSE)
 
  return(c(lo,hi))
}

# Repeated bin search to find the point x in the interval [left, right]
# that satisfies f(x) approx equal to val. If below=TRUE, then we seek
# x such that the above holds and f(x) <= val; else we seek f(x) >= val.

grid.bsearch <- function(left, right, fun, val, gridpts=100, griddepth=1, below=TRUE) {
  n = gridpts
  depth = 1
  print(paste("left ",left, " right ",right, " val ", val))

  while (depth <= griddepth) {
    grid = seq(left,right,length=n)
    vals = fun(grid)
    
    if (below) {
      ii = which(vals >= val)
      print(paste(below," ii "))
      print(ii)
      if (length(ii)==0){
          #print(paste(below," ii empty ", grid[n]))
          return(grid[n])
      }    # All vals < val (shouldn't happen)
      if ((i0=min(ii))==1){
          #print(paste(below," i0==1 ", grid[1]))
          return(grid[1])
      }  # All vals > val (shouldn't happen)
      left = grid[i0-1]
      right = grid[i0]
    }
    
    else {
      ii = which(vals <= val)
      print(paste(below," ii "))
      print(ii)
      if (length(ii)==0) {
          #print(paste(below," ii empty ", grid[1]))
          return(grid[1])   # All vals > val (shouldn't happen)
        }
      if ((i0=max(ii))==n){ 
          #print(paste(below," i0==n ", grid[n]))
          return(grid[n]) # All vals < val (shouldn't happen)
        }
      left = grid[i0]
      right = grid[i0+1]
    }

    depth = depth+1
  }
  #print(paste(below, " bound ", ifelse(below, left, right)))
  return(ifelse(below, left, right))
}
      
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
y = x%*%beta + sigma*rnorm(n)
gfit = glmnet(x,y,standardize=FALSE)
# extract coef for a given lambda; note the 1/n factor!
# (and we don't save the intercept term) fixedLassoInf 7 
lambda = .8
beta = coef(gfit, s=lambda/n, exact=TRUE)[-1]
      
w = beta
a = w != 0


xa = x[,a,drop=F]
xac = x[,!a,drop=F]
xai = pinv(crossprod(xa))
xap = xai %*% t(xa)
za = sign(beta[a])
dz = diag(za)

G = -rbind(
   #   1/lambda * t(xac) %*% P,
   # -1/lambda * t(xac) %*% P,
    -dz %*% xap
      )
u = -c(
   #   1 - t(xac) %*% t(xap) %*% za,
   #   1 + t(xac) %*% t(xap) %*% za,
    -lambda * dz %*% xai %*% za)

 
M = xap
j = 2      
vj = M[j,]
mj = sqrt(sum(vj^2))
vj = vj / mj        # Standardize (divide by norm of vj)
sign_j = sign(sum(vj*y))
print(sign_j)
vj = sign_j * vj
vj
r = poly.pval(y,G,u,vj,1.)
vmat = vj * mj * sign_j
gridrange=c(-100,100)
alpha=0.05
          
a = poly.int(y,G,u,vj,sigma,alpha,gridrange=gridrange,
      flip=(sign_j==-1),bits=NULL)
ci = a$int * mj # Unstandardize (mult by norm of vj)
tailarea = a$tailarea
sd = sigma*sqrt(sum(vmat^2))
print(paste("Ci ",ci," mj ", mj, " sd ",sd))
      
round(ci,3)

[1] 1
[1] 7.364081e-16
  [1] 5.932295e-137 1.663816e-134 4.666386e-132 1.308724e-129 3.670350e-127
  [6] 1.029338e-124 2.886686e-122 8.095269e-120 2.270140e-117 6.365943e-115
 [11] 1.785095e-112 5.005495e-110 1.403523e-107 3.935297e-105 1.103367e-102
 [16] 3.093472e-100  8.672703e-98  2.431328e-95  6.815722e-93  1.910547e-90
 [21]  5.355247e-88  1.500979e-85  4.206697e-83  1.178898e-80  3.303521e-78
 [26]  9.256357e-76  2.593353e-73  7.265027e-71  2.034989e-68  5.699414e-66
 [31]  1.596009e-63  4.468579e-61  1.250903e-58  3.500964e-56  9.795997e-54
 [36]  2.740260e-51  7.663006e-49  2.142145e-46  5.985691e-44  1.671715e-41
 [41]  4.666066e-39  1.301450e-36  3.626802e-34  1.009601e-31  2.806628e-29
 [46]  7.788677e-27  2.156516e-24  5.952676e-22  1.636168e-19  4.469754e-17
 [51]  1.209780e-14  0.000000e+00  0.000000e+00  2.065933e-07  4.486003e-05
 [56]  6.585602e-03  2.551857e-01  9.112382e-01  9.996221e-01  1.000000e+00
 [61]  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.

(array([ 1.9847738 ,  1.07075913,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ]), 0.05496141461582838)


ValueError: zero-size array to reduction operation minimum which has no identity

time: 33.2 ms


In [10]:
from GELMMnet import GELMMnet, kfold, max_l1
import time
start = time.time()
g = kfold(y,x,np.identity(x.shape[1]),nfold=3, alpha_nof=100, ratio_nof=11, centered=True,cpu=4)
end = time.time()
print("CV took", end-start)
g.sigma

Ratios: [ 0.001   0.1009  0.2008  0.3007  0.4006  0.5005  0.6004  0.7003  0.8002
  0.9001  1.    ]
Alphas: [ 0.001    0.01099  0.02098  0.03097  0.04096  0.05095  0.06094  0.07093
  0.08092  0.09091  0.1009   0.11089  0.12088  0.13087  0.14086  0.15085
  0.16084  0.17083  0.18082  0.19081  0.2008   0.21079  0.22078  0.23077
  0.24076  0.25075  0.26074  0.27073  0.28072  0.29071  0.3007   0.31069
  0.32068  0.33067  0.34066  0.35065  0.36064  0.37063  0.38062  0.39061
  0.4006   0.41059  0.42058  0.43057  0.44056  0.45055  0.46054  0.47053
  0.48052  0.49051  0.5005   0.51049  0.52048  0.53047  0.54046  0.55045
  0.56044  0.57043  0.58042  0.59041  0.6004   0.61039  0.62038  0.63037
  0.64036  0.65035  0.66034  0.67033  0.68032  0.69031  0.7003   0.71029
  0.72028  0.73027  0.74026  0.75025  0.76024  0.77023  0.78022  0.79021
  0.8002   0.81019  0.82018  0.83017  0.84016  0.85015  0.86014  0.87013
  0.88012  0.89011  0.9001   0.91009  0.92008  0.93007  0.94006  0.95005
  0.96004  0.9700

5.628747979322303

Sigmahat [ 1.05045344]
pval 5.80989919668e-38 13.647455133 20.6217900366
pval 1.31998155967e-12 11.6684726866 28.0862222087


/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:320: RuntimeWarning: invalid value encountered in double_scalars
  p = (norm.cdf(bb) - norm.cdf(zz)) / (norm.cdf(bb) - norm.cdf(aa))
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:328: RuntimeWarning: overflow encountered in exp
  term1 = np.exp(zz * zz)
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:333: RuntimeWarning: overflow encountered in exp
  term2 = ff(bb) * np.exp(-(np.power(bb, 2) - np.power(zz, 2)) / 2)
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:334: RuntimeWarning: invalid value encountered in double_scalars
  p = (ff(zz) - term2) / (term1 - term2)
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:365: RuntimeWarning: invalid value encountered in greater_equal
  ii = np.where(vals >= val1)[0]
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:366: RuntimeWarning: invalid value encountered in less_equal
  j

pval 0.549457963422 0.313581605049 4.71619772274
pval 0.605716662524 0.253793630022 10.8565308825
pval 0.268841025407 0.254212263274 3.98893673456
pval 0.3648980952 0.110223468909 3.52424279542
pval 0.364292808904 0.0750726793194 2.13362920703
pval 0.893774068483 0.187060064429 9.0390259529
pval 0.520856577917 0.122824114916 3.68328117996


,pval,coef0,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
0.0,5.809899e-38,2.911576,2.898224,18.896700,2.513739,3.472701,0.004922,0.004898,0.154079
1.0,1.319982e-12,2.122611,2.085523,13.803195,1.633319,2.521368,0.004960,0.004755,0.153777
2.0,5.494580e-01,0.141078,0.099045,0.816518,-1.708236,0.579989,0.004972,0.004790,0.172780
3.0,6.057167e-01,0.109437,0.070991,0.699300,-1.773966,0.496583,0.004994,0.004938,0.156495
4.0,2.688410e-01,-0.205650,-0.165374,-1.240091,-0.675115,0.739412,0.004822,0.004941,0.165835
5.0,3.648981e-01,-0.144647,-0.127936,-0.967806,-0.577948,0.821937,0.004879,0.004923,0.149458
6.0,3.642928e-01,0.141072,0.129365,0.908924,-0.888397,0.872561,0.004935,0.004962,0.155208
7.0,8.937741e-01,0.045156,0.017888,0.303971,-6.694897,0.353161,0.004991,0.004887,0.148555
8.0,5.208566e-01,-0.112833,-0.093635,-0.722396,-0.537056,1.298816,0.004774,0.004961,0.156193
